In [17]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import itertools

In [18]:
binarios = [list(map(int, list(bin(i)[2:].zfill(8)))) for i in range(256)]
binarios = np.array(binarios)
# Función para calcular el conjunto Y a partir de los binarios.
def XNOR_bit_a_bit(binario):
    y = binario.copy()
    for i in range(1,8):
        y[i] = not(binario[i] ^ y[i-1])
    return y

# Generar el conjunto Y para cada binario en la lista 'binarios'.
Y_np = np.array([XNOR_bit_a_bit(binario) for binario in binarios])

# Generar el conjunto Y para cada binario en la lista 'binarios' e imprimirlo.
for index, binario in enumerate(binarios):
    Y_test = XNOR_bit_a_bit(binario)
    print(f"Binario: {binario} - Y: {Y_test}")

Binario: [0 0 0 0 0 0 0 0] - Y: [0 1 0 1 0 1 0 1]
Binario: [0 0 0 0 0 0 0 1] - Y: [0 1 0 1 0 1 0 0]
Binario: [0 0 0 0 0 0 1 0] - Y: [0 1 0 1 0 1 1 0]
Binario: [0 0 0 0 0 0 1 1] - Y: [0 1 0 1 0 1 1 1]
Binario: [0 0 0 0 0 1 0 0] - Y: [0 1 0 1 0 0 1 0]
Binario: [0 0 0 0 0 1 0 1] - Y: [0 1 0 1 0 0 1 1]
Binario: [0 0 0 0 0 1 1 0] - Y: [0 1 0 1 0 0 0 1]
Binario: [0 0 0 0 0 1 1 1] - Y: [0 1 0 1 0 0 0 0]
Binario: [0 0 0 0 1 0 0 0] - Y: [0 1 0 1 1 0 1 0]
Binario: [0 0 0 0 1 0 0 1] - Y: [0 1 0 1 1 0 1 1]
Binario: [0 0 0 0 1 0 1 0] - Y: [0 1 0 1 1 0 0 1]
Binario: [0 0 0 0 1 0 1 1] - Y: [0 1 0 1 1 0 0 0]
Binario: [0 0 0 0 1 1 0 0] - Y: [0 1 0 1 1 1 0 1]
Binario: [0 0 0 0 1 1 0 1] - Y: [0 1 0 1 1 1 0 0]
Binario: [0 0 0 0 1 1 1 0] - Y: [0 1 0 1 1 1 1 0]
Binario: [0 0 0 0 1 1 1 1] - Y: [0 1 0 1 1 1 1 1]
Binario: [0 0 0 1 0 0 0 0] - Y: [0 1 0 0 1 0 1 0]
Binario: [0 0 0 1 0 0 0 1] - Y: [0 1 0 0 1 0 1 1]
Binario: [0 0 0 1 0 0 1 0] - Y: [0 1 0 0 1 0 0 1]
Binario: [0 0 0 1 0 0 1 1] - Y: [0 1 0 0 1 0 0 0]


In [19]:
# Convertir los datos en tensores de PyTorch
X = torch.tensor(binarios, dtype=torch.float32)
Y = torch.tensor(Y_np, dtype=torch.float32)

In [20]:
# Definir una red neuronal simple
class Model_qm(nn.Module):
    def __init__(self):
        super(Model_qm, self).__init__()
        self.fc1 = nn.Linear(8, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 8)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [21]:
# Crear una instancia del modelo
q_m_XNOR = Model_qm()
# Definir función de pérdida y optimizador
criterion = nn.BCELoss()
optimizer = optim.Adam(q_m_XNOR.parameters(), lr=0.001)

In [22]:
# Entrenar el modelo (overfitting intencional)
num_epochs = 10000
for epoch in range(num_epochs):
    # Forward pass
    outputs = q_m_XNOR(X)
    # Calcular la pérdida
    loss = criterion(outputs, Y)

    # Backward y optimización
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluación final (opcional, solo para verificar el overfitting)
with torch.no_grad():
    q_m_XNOR.eval()
    predicted_Y = q_m_XNOR(X)
    predicted_Y = (predicted_Y > 0.5).float()  # Convertir las salidas a etiquetas binarias (0 o 1)

    accuracy = (predicted_Y == Y).float().mean().item()
    print(f"Exactitud final del modelo: {accuracy:.4f}")

Epoch [100/10000], Loss: 0.5919
Epoch [200/10000], Loss: 0.4905
Epoch [300/10000], Loss: 0.4082
Epoch [400/10000], Loss: 0.3237
Epoch [500/10000], Loss: 0.2610
Epoch [600/10000], Loss: 0.2093
Epoch [700/10000], Loss: 0.1876
Epoch [800/10000], Loss: 0.1686
Epoch [900/10000], Loss: 0.1342
Epoch [1000/10000], Loss: 0.1116
Epoch [1100/10000], Loss: 0.1001
Epoch [1200/10000], Loss: 0.0947
Epoch [1300/10000], Loss: 0.0920
Epoch [1400/10000], Loss: 0.0904
Epoch [1500/10000], Loss: 0.0889
Epoch [1600/10000], Loss: 0.0819
Epoch [1700/10000], Loss: 0.0591
Epoch [1800/10000], Loss: 0.0351
Epoch [1900/10000], Loss: 0.0207
Epoch [2000/10000], Loss: 0.0130
Epoch [2100/10000], Loss: 0.0086
Epoch [2200/10000], Loss: 0.0060
Epoch [2300/10000], Loss: 0.0044
Epoch [2400/10000], Loss: 0.0034
Epoch [2500/10000], Loss: 0.0027
Epoch [2600/10000], Loss: 0.0021
Epoch [2700/10000], Loss: 0.0017
Epoch [2800/10000], Loss: 0.0015
Epoch [2900/10000], Loss: 0.0012
Epoch [3000/10000], Loss: 0.0011
Epoch [3100/10000],

In [23]:
# Guardar los pesos del modelo en un archivo .pth
torch.save(q_m_XNOR.state_dict(), 'q_m_XNOR.pth')
print("Los pesos del modelo se han guardado")

Los pesos del modelo se han guardado
